In [195]:
#notebook for testing the search of song in a wav file

## Load everything
%matplotlib inline
import pdb
import sys
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib
import scipy.signal as sg
import math
import scipy as sp
import socket
import os

matplotlib.style.use('ggplot')

# Set up Computer and paths
comp_name=socket.gethostname()
print 'Computer: ' + comp_name
if  comp_name == 'chim':
    sys.path.append('C:\Users\GentnerLab\Documents\Experiment\scripts\sound_tools')
    experiment_folder = os.path.join('C:\Users\GentnerLab\Documents\Experiment')
    r = os.path.join('C:\Users\GentnerLab\Documents\Experiment')
    raw_data_folder = os.path.join(experiment_folder, 'raw_data')

if 'lintu' in comp_name:
    sys.path.append('/mnt/cube/earneodo/repos/soundflow/sound_tools')
    experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')
    raw_data_folder = os.path.join(experiment_folder, 'raw_data')

if 'lookfar' in comp_name:
    sys.path.append('/Users/zeke/repos/soundflow/sound_tools')
    experiment_folder = os.path.join('/Volumes/gentner/earneodo/bci_zf/')
    raw_data_folder = os.path.join(experiment_folder, 'raw_data')

#grab soundtools
from soundtools import WavData, Chunk
    
#file structure
bird_id = 'z003'
sess = 1
rec = 'a'
run = 5
bout = '2251_869'

#load the bout and the whole sound file for searching
raw_file_folder = os.path.join(raw_data_folder, bird_id, str(sess).zfill(3))
raw_file_name = rec + '_' + str(run).zfill(2) + '_song.wav'
raw_file_path = os.path.join(raw_file_folder, raw_file_name)

#the whole sound
all_sound = WavData(raw_file_path)

#the bout
if bout is not None:
    raw_file_folder = os.path.join(raw_file_folder, 'bouts')
    raw_file_name = rec + '_' + str(run).zfill(2) + '_song_' + bout + '.wav'

raw_file_path = os.path.join(raw_file_folder, raw_file_name)
bout_sound = WavData(raw_file_path)

Computer: lookfar.local


In [196]:
#functions that should be in a package
def rms(x):
    return np.linalg.norm(x)

def mad(x):
    med = np.median(x)
    dev = np.abs(x - np.median(x))
    return np.median(dev)

def identity(x):
    return x

#decorator for getting the rms after applying a filter to a segment
def rms_after_filter(filter_func):
    def out_of_filtered(*args, **kwargs):
        #print "Arguments were: %s, %s" % (args, kwargs)
        #plt.plot(np.transpose(filter_func(*args, **kwargs)))
        return np.linalg.norm(filter_func(*args, **kwargs))
    return out_of_filtered

In [197]:
# preparations and parameters
# filter parameters and filter function
#filter the signal
s_f = all_sound.s_f

#The band where we expect most of the energy of zf song to be in
filt_lo = 10000 #Hz
filt_hi = 400 #Hz
hp_b, hp_a = sg.butter(4, filt_hi/(s_f/2.), btype='high')
lp_b, lp_a = sg.butter(4, filt_lo/(s_f/2.), btype='low')


def band_pass_filter(chunk, hp_b, hp_a, lp_b, lp_a):    
    chunk_hi = sg.filtfilt(hp_b, hp_a, chunk)
    chunk_filt = sg.filtfilt(lp_b, lp_a, chunk_hi)
    return chunk_filt

@rms_after_filter
def rms_band_pass_filter(chunk, hp_b, hp_a, lp_b, lp_a):
    #plt.plot(np.transpose(chunk))
    chunk_hi = sg.filtfilt(hp_b, hp_a, chunk)
    chunk_filt = sg.filtfilt(lp_b, lp_a, chunk_hi)
    return chunk_filt

@rms_after_filter
def rms_identity(chunk):
    return(chunk)

In [198]:
#Parameters of the search
bit_size = 100 # ms; size of sound bits
refractory = 5 # bits; refractory period in bits
bits_bout = 5 # bits; how many bouts together is considered a bout
threshold = 5 # threshold in mads (median deviation of the median)

bit_size_sample = np.int(bit_size*all_sound.s_f*0.001)
all_starts = np.arange(0, all_sound.n_samples - bit_size_sample, bit_size_sample)

all_rms = all_sound.apply_repeated(all_starts, bit_size_sample, rms_band_pass_filter, hp_b, hp_a, lp_b, lp_a)

thresh_rms = np.median(all_rms) + threshold * mad(all_rms)

print np.median(all_rms)
print mad(all_rms)
print thresh_rms

KeyboardInterrupt: 

In [ ]:
supra_bits = np.where(all_rms > thresh_rms)[0]
plt.plot(np.arange(supra_bits.size), supra_bits, '*')


for bit, start in enumerate(supra_bits, all_starts[supra_bits]):
    #check if it is the beginning of a chunk
    if diff(supra_bits[bit:bit+bits_bout+1] == 0)
        begin = bit
        

In [ ]:
# Starts of probable bouts
possible_bouts = np.where(np.diff(supra_bits, bits_bout)==0)[0]
firsts = possible_bouts[np.where(np.diff(possible_bouts,1)>10)[0]+1]
firsts = np.insert(firsts, 0, possible_bouts[0])
lasts = possible_bouts[np.where(np.diff(possible_bouts,1)>10)[0]-1]
lasts = np.insert(lasts, lasts.size, possible_bouts[-1])

assert(firsts.size == lasts.size)
segments = np.array([firsts, lasts])
more = 2
i=12
start = all_starts[segments[0, i]] - more * bits_bout 
end = all_starts[segments[1, i]] + more * bits_bout
start = start if start > 0 else 0
end = end if end < all_sound.n_samples else all_sound.n_samples
sg_chunk = Chunk(all_sound, segment = [start, end])
plt.plot(np.transpose(sg_chunk.data))

In [ ]:
# Starts of probable bouts
possible_bouts = np.where(np.diff(supra_bits, bits_bout)==0)[0]

#get begginings of bits_bout chunks above threshold
firsts = possible_bouts[np.where(np.diff(possible_bouts,1)>1)[0]+1]
firsts = np.insert(firsts, 0, possible_bouts[0])

lasts = possible_bouts[np.where(np.diff(possible_bouts,1)>1)[0]] + bits_bout
lasts = np.insert(lasts, lasts.size, possible_bouts[-1]) 

#gather segments that are less than refractory apart
together = np.where((firsts[1:] - lasts[:-1]) < refractory)[0]

lasts = np.delete(lasts, together)
firsts = np.delete(firsts, together +1)
assert(firsts.size == lasts.size)

segments = np.array([all_starts[supra_bits[firsts]], all_starts[supra_bits[lasts]]])
assert(np.all(np.diff(segments, axis=0)[0]>0))


In [ ]:
#Make a chunk with each segment and record it to a file
more = 2 #how many bits before and after to save
n_seg = segments.shape[1]

bout_file_name = rec + '_' + str(run).zfill(2) + 'bouts.wav'
bout_file_path = os.path.join(raw_file_folder, bout_file_name)

bout_file_path = '/Users/zeke/bout_test.wav'

import wave
import struct

bouts_out = wave.open(bout_file_path, 'w')
bouts_out.setparams((all_sound.n_chans, 2, all_sound.s_f, 0, 'NONE', 'not compressed'))

for seg in segments.T:
    seg = seg + np.array([-more, more])* bit_size_sample

    start = seg[0] if seg[0] > 0 else 0
    end = seg[1] if seg[1] < all_sound.n_samples else all_sound.n_samples
    #print start, end
    sg_chunk = Chunk(all_sound, segment = [start, end])
    # save the chunk to a data file
    data_out = np.array(sg_chunk.data, dtype=np.int16)[0]
    packed_data = struct.pack('<' + str((end - start)) + 'h', *data_out)
    bouts_out.writeframes(data_out)

bouts_out.close()

In [ ]:
'<' + str((end - start)) + 'h'

In [ ]:
data_out.shape

In [ ]:
firsts = possible_bouts[np.where(np.diff(possible_bouts,1)>1)[0]+1]
firsts = np.insert(firsts, 0, possible_bouts[0])
print firsts

In [ ]:
sg_chunk = Chunk(all_sound, segment=[0, 10000]) 

In [ ]:
lasts = possible_bouts[np.where(np.diff(possible_bouts,1)>1)[0]] + bits_bout
lasts = np.insert(lasts, lasts.size, possible_bouts[-1]) 
print lasts

In [ ]:
firsts[1:] - lasts[:-1]

In [ ]:
together = np.where((firsts[1:] - lasts[:-1]) < refractory)[0]
print together

In [ ]:
lasts = np.delete(lasts, together)

In [ ]:
firsts = np.delete(firsts, together +1)

In [ ]:
possible_bouts = np.where(np.diff(supra_bits, 5)==0)[0]
pb = possible_bouts[80:120]
print pb


In [ ]:
bstarts = all_starts[supra_bits[firsts]]

In [ ]:
bends = all_starts[supra_bits[lasts]]

In [ ]:
bends - bstarts

In [ ]:
pb[np.where(np.diff(pb)>1)]

In [ ]:
supra_bits[starts[0]:ends[0]]

In [ ]:
j = 8
chu = Chunk(all_sound, segment=[bstarts[j], bends[j]])
chu.apply_filter(band_pass_filter, hp_b, hp_a, lp_b, lp_a)
plt.plot(np.transpose(chu.data))

In [ ]:
possible_bouts